<a href="https://colab.research.google.com/github/dean-baldwin/guitarists/blob/main/bluesartists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyvis

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyvis.network import Network
import networkx as nx
import IPython

In [3]:
# load the guitarist and influencer dataset
path = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSvZjEr8ctmljiJsE5OTJV6bG4PiDwNX56exZOSoA5skuwUz_orVftdFQWfEYw2HQuG0aIfFbiL-NAM/pub?gid=971011409&single=true&output=csv'
df = pd.read_csv(path)

weights = df['influence'].value_counts().to_frame()
weights.loc['unavailable'] = 0

In [4]:
path_to_image = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSvZjEr8ctmljiJsE5OTJV6bG4PiDwNX56exZOSoA5skuwUz_orVftdFQWfEYw2HQuG0aIfFbiL-NAM/pub?gid=2045385092&single=true&output=csv'
image_df = pd.read_csv(path_to_image)

In [5]:
weight = []
for index, row in df.iterrows():
  pWeight = 0
  iWeight = 0
  if row['guitarist'] in weights.index:
    pWeight = weights.loc[row['guitarist']]['influence']

  if row['influence'] in weights.index:
    iWeight = weights.loc[row['influence']]['influence']

  weight.append(pWeight + iWeight) 

df['weight'] = weight

In [6]:
df['guitarist'].nunique()

95

In [7]:
gtr_df = df.merge(image_df[['guitarist','image']], on="guitarist", how="left")
gtr_df["image"] = gtr_df["image"]

In [8]:
gtr_df.loc[df['guitarist']=='Pete Seeger']

,guitarist,influence,weight,image
144,Pete Seeger,unavailable,1,https://deanbaldwinmusic.com/guitarists/petese...


In [9]:
gtr_net = Network(height='1000px', width='100%', bgcolor='white', font_color='#222222', heading="Lines of Influence")

# set the physics layout of the network
gtr_net.barnes_hut()

sources = gtr_df['guitarist']
targets = gtr_df['influence']
weights = gtr_df['weight']
image = gtr_df['image']

edge_data = zip(sources, targets, image, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    img = e[2]
    w = e[3]
  
    gtr_net.add_node(src, src, title=src, labelHighlightBold=True, shape='image', image = img)
    gtr_net.add_node(dst, dst, title=dst)
    edge_title = '<b>' + src + "</b></br>was infuenced by</br>" + dst
    gtr_net.add_edge(src, dst, value=w, arrowStrikethrough=False, title=edge_title)
    

neighbor_map = gtr_net.get_adj_list()

# add neighbor data to node hover data
"""
for node in gtr_net.nodes:
    node['title'] += ' Influences:<br>' + '<br>'.join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])
"""


gtr_net.set_options(
"""
var options = {
  "nodes": {
    "color": {
      "border": "rgba(131,138,165,0.47)",
      "background": "rgba(137,138,155,0.78)"
    },
    "shadow": {
      "enabled": true
    }
  },
  "font": {
    "size": 50,
    "face": "verdana",
    "strokeColor": "rgba(190,168,255,1)"
  },
  "edges": {
    "arrows": {
      "to": {
        "enabled": true
      }
    },
    "color": {
      "color": "rgba(199,205,214,0.31)",
      "highlight": "rgba(104,133,214,1)",
      "hover": "rgba(129,186,255,1)",
      "inherit": false
    },
    "hoverWidth": 2
  },
  "interaction": {
    "hover": true,
    "multiselect": true
  },
  "manipulation": {
    "enabled": false,
    "initiallyActive": false
  },
  "physics": {
    "forceAtlas2Based": {
      "springLength": 100
    },
    "minVelocity": 0.75,
    "solver": "forceAtlas2Based"
  }
}
"""
)

#gtr_net.show_buttons(filter_=None)
gtr_net.show('/content/drive/MyDrive/guitarists.html')

FileNotFoundError: ignored

In [ ]:
IPython.display.HTML(filename='/content/drive/MyDrive/guitarists.html')